# Equation Solver using False Position Method

This program numerically solves equations using False Position method.
---
- Enter the equation to solve.
- Take a guess to start fake position.
- The program will find the root near to the guess.
---

## Importing libraries
- `sympy` for converting user input to a python lambda function.
- `math` for the use of sympy.
- `time` for setting timeout.
- `sys` for exiting the program.

In [1]:
import sympy as sp
import math
import time
import sys

## Declaring variables:
- `pos`: a value of x for which the function evaluates to a positive value.
- `neg`: a value of x for which the function evaluates to a negative value.
- `res`: a root/approx. root of the equation.
- `x`: a sympy symbol for using in equation.

In [2]:
pos, neg, res = None, None, None
x= sp.symbols('x')

## Defining Equation
### Taking input and converting to lambda

Ask the user for the equation. Check if it is an equation or a function. In case of a function take it as left hand side of the equation and 0 as the right hand side. And for an equation split and perse the left and right hand separately. Then define the function to be `f(x)= lhs - rhs`. Then we will convert the sympy function to a python labda for easy handeling. If we can't parse the function then ask again until a persable input is obtained.

In [3]:
sp_func = None

while sp_func is None:
    func_input = input("Please enter the equation to solve f(x)=g(x) (e.g. sin(x) + x**2 = 0): ")
    try:
        if "=" in func_input:
            lhs_str, rhs_str = func_input.split("=")
            lhs, rhs = sp.sympify(lhs_str), sp.sympify(rhs_str)
        else:
            lhs = sp.sympify(func_input)
            rhs = 0
            
        sp_func = lhs - rhs
        func = sp.lambdify(x, sp_func, modules=["math"])
        
    except Exception as e:
        print("Error parsing function:", e)

print("Function parsed successfully:", sp_func)

Function parsed successfully: x*exp(x) - 1


### Taking the first guess as input
An equation can have multiple roots. To find the root near a desired value of `x` we need to calculate around that point. This cell asks the user to make the first guess and store it into the variable `strt`. Moreover it checks if the guess itself is the root. If so then it assigns the value to the `res` variable.

In [4]:
strt = None
while strt is None:
    strt = float(input("Enter a guess around which you want to solve the equation: "))
    try:
        fv = func(strt)
        pos, neg = strt, strt
        if fv == 0:
            res = strt
        break

    except Exception as e:
        print("Error evaluating function at the guess:", e)
        strt = None

## False position Method
### Finding x for a positive and a negative function value
This cell declares to variable `i` and initiates a loop. While the functional value at `pos` is negative or the functional value at `neg` is positive, we will try to find the value for `pos` and `neg`. As by definition they are the value of `x` at which the function evaluates to a positive and a negative value respectively. For each iteration we check if the functional value at a point that is `i` more than the starting guess is positive or negative. If positive and we haven't find the `pos` value then we assign it this value. Same goes for the `neg`. We will also check for the value of `x` that is `i` less than the starting guess. If somehow for any value of `x` the function evaluatesto zero then set it as the root and exit the loop.

In [5]:
s_time = time.time()
rs_time = time.time()
increment = 1
i = 0
while (func(pos) < 0 or func(neg) > 0) and res is None:
    try:
        fv = func(strt + i)
        if fv > 0 and func(pos) <= 0:
            pos = strt + i
        elif fv < 0 and func(neg) >= 0:
            neg = strt + i
        elif fv == 0:
            res = strt + i
            break
    except Exception as e:
        pass
    try:
        fv = func(strt - i)
        if fv > 0 and func(pos) <= 0:
            pos = strt - i
        elif fv < 0 and func(neg) >= 0:
            neg = strt - i
        elif fv == 0:
            res = strt - i
            break
    except Exception as e:
        pass
    if time.time() - s_time > 15:
        i = 0
        s_time = time.time()
        increment = increment / 10
        print("Switching to finer increment search...")
        print("New increment:", increment)
    if time.time() - rs_time > 120:
        print(
            "Could not find suitable interval in time limit. Please try again with a different initial guess."
        )
        print("Function may not have a root near the guess or may be undefined in that region.")
        print("Exiting...")
        sys.exit(1)
        break
    i += increment


print("Found interval: f(", neg, ") =", func(neg), ", f(", pos, ") =", func(pos))

print(res)

Found interval: f( 0.0 ) = -1.0 , f( 1.0 ) = 1.718281828459045
None


### Perform false position

Now we will perform the false position. At first we have to declare a variable `newx` that will store the value of our next guess for `x` temporarily. While we don't find a value for `res` or result, we will continue false position loop. At first we will check if the `newx` is going to be the same as the previous which will imply that the value of `x` will no longer change or we have reached an recursion. Usually it means wehave found the root for the precision of floats python gives us. In that case we will exit the loop. Otherwise we will assign the `newx` variable to the next guess we obtain from the current values of `pos` and `neg`. Then we will evaluate the function at the point `newx`. If the functional value at `newx` is positive we will assign the `pos` with this value of `newx`. And for negative we will assign `neg`. And in case the funcitonal value is `0`, it's a root for the equation thus we can assign `res` to this value of `newx` and exit the loop.

**Note:** We don't usually get `0` as the functional value but an extremely small value.

In [6]:
avg = 0
newx = None
res=None

while res is None:
    if (newx is not None) and (
        newx == (pos * func(neg) - neg * func(pos)) / (func(neg) - func(pos))
    ):
        res = newx
        break

    newx = (pos * func(neg) - neg * func(pos)) / (func(neg) - func(pos))

    fv = func(newx)
    print("f(", newx, ") =", fv)
    if fv > 0:
        pos = newx
        print("Root exist between", neg, "and", pos)
    elif fv < 0:
        neg = newx
        print("Root exist between", neg, "and", pos)
    else:
        res = newx
        break

print("Approximate root found at x =", res)

f( 0.36787944117144233 ) = -0.46853639461338437
Root exist between 0.36787944117144233 and 1.0
f( 0.5033143321329856 ) = -0.16742007618244814
Root exist between 0.5033143321329856 and 1.0
f( 0.547412050944398 ) = -0.05364869147900564
Root exist between 0.547412050944398 and 1.0
f( 0.5611150437947322 ) = -0.016575372928013477
Root exist between 0.5611150437947322 and 1.0
f( 0.5653082891037876 ) = -0.005062903191810797
Root exist between 0.5653082891037876 and 1.0
f( 0.5665853418084091 ) = -0.001541031941459381
Root exist between 0.5665853418084091 and 1.0
f( 0.5669736991488938 ) = -0.0004685533567156952
Root exist between 0.5669736991488938 and 1.0
f( 0.5670917476619046 ) = -0.00014241808542680978
Root exist between 0.5670917476619046 and 1.0
f( 0.5671276258593797 ) = -4.328408802534156e-05
Root exist between 0.5671276258593797 and 1.0
f( 0.5671385297830954 ) = -1.3154621078759199e-05
Root exist between 0.5671385297830954 and 1.0
f( 0.5671418436079488 ) = -3.9978311299204705e-06
Root ex

## Print the result
The above operations will ensure that we find a root or approximate root at the variable `res`. So it is safe to print the value of `res` and call it a root of the equation.

In [7]:

print("Approximate root at x =", res)

Approximate root at x = 0.5671432904097838
